<a href="https://colab.research.google.com/github/nithinbadicodes/food-101/blob/master/tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this notebook, we're going to cover some of the most fundamental concepts of tensors using TensorFlow

More specifically, we're going to cover:
* Introduction to tensors
* Getting information from tensors
* Manipulating tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up your regular Python functions)
* Using GPUs with TensorFlow
* Exercises

## Introduction to Tensors

In [2]:
# Import Tensorflow
import tensorflow as tf
print(tf.__version__)

2.18.0


In [3]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [6]:
a = tf.Variable(7)
a.numpy()

np.int32(7)

### Creating random tensors

Random tensors are tensors of abitrary values

array('false', dtype='<U5')